In [4]:
import json
import os
import math
from PIL import Image
import shutil

# liczba punktów do aproksymacji elipsy
N_POINTS = 50

# mapowanie klas -> ID
CLASS_MAP = {"Pupil": 0}

def ellipse_to_polygon(cx, cy, rx, ry, rotation=0, n_points=N_POINTS):
    """Zamienia elipsę na listę punktów."""
    points = []
    angle_rad = math.radians(rotation)
    for i in range(n_points):
        theta = 2 * math.pi * i / n_points
        x = rx * math.cos(theta)
        y = ry * math.sin(theta)

        # rotacja elipsy
        xr = x * math.cos(angle_rad) - y * math.sin(angle_rad)
        yr = x * math.sin(angle_rad) + y * math.cos(angle_rad)

        points.append((cx + xr, cy + yr))
    return points

# plik z Label Studio
EXPORT_JSON = "D:/AAA/Nowy folder/Outputs/project-12-at-2025-09-10-11-40-f33b0828/project-12-at-2025-09-10-12-08-f33b0828.json"
# katalog, gdzie będą zapisane dane YOLO
OUTPUT_DIR = "D:/AAA/Nowy folder/Outputs/project-12-at-2025-09-10-11-40-f33b0828/yolo_dataset"

os.makedirs(os.path.join(OUTPUT_DIR, "images/train"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, "labels/train"), exist_ok=True)

# wczytaj JSON
with open(EXPORT_JSON, "r", encoding="utf-8") as f:
    tasks = json.load(f)

for task in tasks:
    img_path = task["data"]["image"]  # np. "/data/upload/12/xxx.png"
    img_filename = os.path.basename(task["file_upload"])  # faktyczna nazwa pliku

    # Wymiary obrazu
    w = task["annotations"][0]["result"][0]["original_width"]
    h = task["annotations"][0]["result"][0]["original_height"]

    # kopiujemy obraz (musisz mieć go w tym samym katalogu, co export.json!)
    src_img_path = img_filename
    dst_img_path = os.path.join(OUTPUT_DIR, "images/train", img_filename)
    if os.path.exists(src_img_path):
        shutil.copy(src_img_path, dst_img_path)

    # plik z etykietami
    label_path = os.path.join(
        OUTPUT_DIR, "labels/train", os.path.splitext(img_filename)[0] + ".txt"
    )

    with open(label_path, "w") as lf:
        for ann in task["annotations"][0]["result"]:
            if ann["type"] == "ellipselabels":
                val = ann["value"]
                cx = val["x"] / 100.0 * w
                cy = val["y"] / 100.0 * h
                rx = val["radiusX"] / 100.0 * w
                ry = val["radiusY"] / 100.0 * h
                rotation = val.get("rotation", 0)

                polygon = ellipse_to_polygon(cx, cy, rx, ry, rotation)

                # normalizacja punktów
                norm_points = []
                for (px, py) in polygon:
                    norm_points.append(px / w)
                    norm_points.append(py / h)

                class_name = val["ellipselabels"][0]
                class_id = CLASS_MAP[class_name]

                line = f"{class_id} " + " ".join([f"{p:.6f}" for p in norm_points])
                lf.write(line + "\n")


In [5]:
#Stworzenie podziału na dane uczące i walidacyjne
import os, shutil, random, glob
from pathlib import Path

src_img = Path("D:/AAA/Nowy folder/Outputs/project-12-at-2025-09-10-11-40-f33b0828/images")
src_lab = Path("D:/AAA/Nowy folder/Outputs/project-12-at-2025-09-10-11-40-f33b0828/labels")
out = Path("D:/AAA/Nowy folder/Outputs/project-12-at-2025-09-10-11-40-f33b0828/yolo_pupil_seg"); random.seed(42)
for p in ['images/train','images/val','labels/train','labels/val']:
    (out/p).mkdir(parents=True, exist_ok=True)

imgs = [Path(p) for p in glob.glob(str(src_img/'*'))]
random.shuffle(imgs); n = int(0.8*len(imgs))
splits = {'train': imgs[:n], 'val': imgs[n:]}

for split, files in splits.items():
    for img in files:
        lbl = src_lab/(img.stem + ".txt")
        shutil.copy2(img, out/'images'/split/img.name)
        if lbl.exists(): shutil.copy2(lbl, out/'labels'/split/lbl.name)

In [6]:
from ultralytics import YOLO
model = YOLO('yolo11n-seg.pt')  # lub yolov8n-seg.pt
model.train(data='D:/AAA/Nowy folder/Outputs/project-12-at-2025-09-10-11-40-f33b0828/data.yaml', imgsz=400, epochs=100, batch=16, device=0, workers=8, project='runs', name='pupil-seg-11')

New https://pypi.org/project/ultralytics/8.3.197 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.177  Python-3.11.9 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/AAA/Nowy folder/Outputs/project-12-at-2025-09-10-11-40-f33b0828/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=400, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale

train: Scanning D:\AAA\Nowy folder\Outputs\project-12-at-2025-09-10-11-40-f33b0828\yolo_pupil_seg\labels\train... 29 images, 0 backgrounds, 0 corrupt: 100%|██████████| 29/29 [00:00<00:00, 286.05it/s]

train: New cache created: D:\AAA\Nowy folder\Outputs\project-12-at-2025-09-10-11-40-f33b0828\yolo_pupil_seg\labels\train.cache


val: Fast image access  (ping: 0.10.0 ms, read: 8.61.2 MB/s, size: 79.7 KB)


val: Scanning D:\AAA\Nowy folder\Outputs\project-12-at-2025-09-10-11-40-f33b0828\yolo_pupil_seg\labels\val... 8 images, 0 backgrounds, 0 corrupt: 100%|██████████| 8/8 [00:00<00:00, 254.59it/s]

...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.97it/s]


                   all          8          8      0.994          1      0.995      0.845      0.994          1      0.995      0.786
Speed: 0.3ms preprocess, 6.3ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs\pupil-seg-113


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000016B02A06010>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

In [7]:
from ultralytics import YOLO
import cv2

model = YOLO(r"D:\AAA\Nowy folder\pupiltracking\runs\pupil-seg-113\weights\best.pt")
results = model.predict(
    source=r"D:\AAA\Nowy folder\Outputs\Test",  # może być plik lub folder
    conf=0.35, max_det=1, device=0, save=True, imgsz=320
)


image 1/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.25.48.png: 320x320 1 Pupil, 11.0ms
image 2/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.08.png: 320x320 1 Pupil, 10.6ms
image 3/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.18.png: 320x320 1 Pupil, 11.5ms
image 4/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.22.png: 320x320 1 Pupil, 11.7ms
image 5/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.48.png: 320x320 1 Pupil, 11.6ms
image 6/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.42.36.png: 320x320 1 Pupil, 11.0ms
image 7/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.43.24.png: 320x320 1 Pupil, 15.7ms
image 8/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.43.46.png: 320x320 1 Pupil, 11.1ms
image 9/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.43.50.png: 320x320 1 Pupil, 15.3ms
image 10/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.43.54.png: 320x320 1 Pupil, 11.7ms
image 11/44 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.43.58.png: 320x320 1

In [14]:
from ultralytics import YOLO
import cv2
import os
import glob
import numpy as np

# Ścieżka wejściowa
input_dir = r"D:\AAA\Nowy folder\Outputs\Test"

# Wczytanie modelu segmentacyjnego YOLO
model = YOLO(r"D:\AAA\Nowy folder\pupiltracking\runs\pupil-seg-11\weights\best.pt")

# Pobranie plików graficznych
image_files = glob.glob(os.path.join(input_dir, "*.jpg")) + \
              glob.glob(os.path.join(input_dir, "*.png")) + \
              glob.glob(os.path.join(input_dir, "*.jpeg"))

for img_path in image_files:
    img = cv2.imread(img_path)

    # Detekcja
    results = model.predict(
        source=img_path,
        conf=0.35, max_det=1, device=0, save=False, imgsz=320
    )

    for r in results:
        if r.masks is not None:
            masks = r.masks.xy  # lista poligonów (każdy to tablica Nx2)
            for poly in masks:
                # Konwersja do int32 dla OpenCV
                pts = np.array(poly, dtype=np.int32)
                pts = pts.reshape((-1, 1, 2))

                # Rysowanie poligonu
                cv2.polylines(img, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

                # Obliczanie środka poligonu (momenty geometryczne)
                M = cv2.moments(pts)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                else:
                    cx, cy = 0, 0

                # Rysowanie środka
                cv2.circle(img, (cx, cy), 5, (0, 0, 255), -1)
                cv2.putText(img, f"({cx},{cy})", (cx + 10, cy - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # Wyświetlanie obrazu
    cv2.imshow("Poligon + środek", img)
    key = cv2.waitKey(0)
    if key == 27:  # ESC → przerwij
        break

cv2.destroyAllWindows()



image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.25.48.png: 320x320 1 Pupil, 78.9ms
Speed: 1.7ms preprocess, 78.9ms inference, 3.5ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.08.png: 320x320 1 Pupil, 106.9ms
Speed: 2.3ms preprocess, 106.9ms inference, 7.9ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.18.png: 320x320 1 Pupil, 17.2ms
Speed: 1.5ms preprocess, 17.2ms inference, 4.6ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.22.png: 320x320 1 Pupil, 11.9ms
Speed: 1.2ms preprocess, 11.9ms inference, 5.1ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.48.png: 320x320 1 Pupil, 18.6ms
Speed: 1.1ms preprocess, 18.6ms inference, 4.3ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12

In [13]:
from ultralytics import YOLO
import cv2
import os
import glob
import numpy as np

# Ścieżka wejściowa
input_dir = r"D:\\AAA\\Nowy folder\\Outputs\\Test"

# Wczytanie modelu segmentacyjnego YOLO
model = YOLO(r"D:\\AAA\\Nowy folder\\pupiltracking\\runs\\pupil-seg-11\\weights\\best.pt")

# Pobranie plików graficznych
image_files = glob.glob(os.path.join(input_dir, "*.png")) 

for img_path in image_files:
    img = cv2.imread(img_path)

    # Detekcja
    results = model.predict(
        source=img_path,
        conf=0.35, max_det=1, device=0, save=False, imgsz=320
    )

    for r in results:
        if r.masks is not None:
            masks = r.masks.xy
            for poly in masks:
                pts = np.array(poly, dtype=np.int32)
                pts_reshaped = pts.reshape((-1, 1, 2))

                # Rysowanie poligonu
                cv2.polylines(img, [pts_reshaped], isClosed=True, color=(0, 255, 0), thickness=2)

                # Środek geometryczny
                M = cv2.moments(pts)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])
                else:
                    cx, cy = 0, 0

                # Znalezienie punktów przecięcia poziomej linii y=cy z krawędziami poligonu
                intersections = []
                for i in range(len(pts)):
                    x1, y1 = pts[i]
                    x2, y2 = pts[(i + 1) % len(pts)]  # następny punkt (zamknięty poligon)

                    # Sprawdzamy, czy odcinek przecina linię y=cy
                    if (y1 <= cy <= y2) or (y2 <= cy <= y1):
                        if y1 != y2:  # unikamy dzielenia przez zero
                            t = (cy - y1) / (y2 - y1)
                            x_int = int(x1 + t * (x2 - x1))
                            intersections.append((x_int, cy))

                if len(intersections) >= 2:
                    # Wybieramy najbardziej skrajne punkty
                    p_left = min(intersections, key=lambda p: p[0])
                    p_right = max(intersections, key=lambda p: p[0])
                    diameter_x = abs(p_right[0] - p_left[0])

                    # Rysowanie linii średnicy przez środek
                    cv2.line(img, p_left, p_right, (255, 0, 0), 2)
                    cv2.putText(img, f"{diameter_x}px",
                                ((p_left[0] + p_right[0]) // 2, cy + 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                # Rysowanie środka
                cv2.circle(img, (cx, cy), 5, (0, 0, 255), -1)
                cv2.putText(img, f"({cx},{cy})", (cx + 10, cy - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # Wyświetlanie
    cv2.imshow("image", img)
    key = cv2.waitKey(0)
    if key == 27:
        break

cv2.destroyAllWindows()



image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.25.48.png: 320x320 1 Pupil, 44.0ms
Speed: 1.7ms preprocess, 44.0ms inference, 3.7ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.08.png: 320x320 1 Pupil, 21.0ms
Speed: 1.2ms preprocess, 21.0ms inference, 4.7ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.18.png: 320x320 1 Pupil, 26.3ms
Speed: 1.7ms preprocess, 26.3ms inference, 5.9ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.22.png: 320x320 1 Pupil, 24.8ms
Speed: 1.5ms preprocess, 24.8ms inference, 8.3ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-13.26.48.png: 320x320 1 Pupil, 62.5ms
Speed: 2.0ms preprocess, 62.5ms inference, 5.7ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 D:\AAA\Nowy folder\Outputs\Test\2025-08-12-1

In [1]:
import torch
print(torch.cuda.is_available())  # powinno być True
print(torch.cuda.get_device_name(0))  # powinno pokazać GTX 1050 Ti


True
NVIDIA GeForce GTX 1050 Ti
